In [1]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import missingno         as msno
import math
import gc
from tqdm import tqdm_notebook as tqdm
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

%matplotlib inline
sns.set(rc={'figure.figsize':(11,8)})
sns.set(style="whitegrid")

# Read datasets

In [2]:
path = Path('/run/media/javi/NVMe/Datasets/Kaggle/ashrae-energy-prediction')
! ls {path}

building_metadata.csv  test.csv   weather_test.csv
sample_submission.csv  train.csv  weather_train.csv


In [3]:
%%time
metadata_df       = pd.read_csv(path / 'building_metadata.csv')
train_df          = pd.read_csv(path / 'train.csv',         parse_dates=['timestamp'])
test_df           = pd.read_csv(path / 'test.csv',          parse_dates=['timestamp'])
weather_train_df  = pd.read_csv(path / 'weather_train.csv', parse_dates=['timestamp'])
weather_test_df   = pd.read_csv(path / 'weather_test.csv',  parse_dates=['timestamp'])
sample_submission = pd.read_csv(path / 'sample_submission.csv')

CPU times: user 17.5 s, sys: 2.81 s, total: 20.3 s
Wall time: 21.4 s


# Reduce mem usage

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


metadata_df      = reduce_mem_usage(metadata_df)
train_df         = reduce_mem_usage(train_df)
test_df          = reduce_mem_usage(test_df)
weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df  = reduce_mem_usage(weather_test_df)

Mem. usage decreased to  0.03 Mb (60.3% reduction)
Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)


# Align timestamps (high temperature will be 14:00)

In [5]:
weather = pd.concat([weather_train_df,weather_test_df],ignore_index=True)
weather_key = ['site_id', 'timestamp']

temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

# calculate ranks of hourly temperatures within date/site_id chunks
temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')

# create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

# Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
site_ids_offsets.index.name = 'site_id'

def timestamp_align(df):
    df['offset'] = df.site_id.map(site_ids_offsets)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned']
    return df

In [6]:
weather_train_df = timestamp_align(weather_train_df)
weather_test_df  = timestamp_align(weather_test_df)

In [7]:
del weather 
del df_2d
del temp_skeleton
del site_ids_offsets
gc.collect()

75

# Fill NaNs in weather data by interpolation

In [8]:
def check_missing(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent missing'])

print("Train")
display(check_missing(weather_train_df))
print("Test")
display(check_missing(weather_test_df))

Train


,Total,Percent missing
cloud_coverage,69173,49.489529
precip_depth_1_hr,50289,35.979052
sea_level_pressure,10618,7.596603
wind_direction,6268,4.484414
wind_speed,304,0.217496
dew_temperature,113,0.080845
air_temperature,55,0.039350
offset,0,0.000000
timestamp,0,0.000000
site_id,0,0.000000


Test


,Total,Percent missing
cloud_coverage,140448,50.658808
precip_depth_1_hr,95588,34.478057
sea_level_pressure,21265,7.670167
wind_direction,12370,4.461790
wind_speed,460,0.165919
dew_temperature,327,0.117947
air_temperature,104,0.037512
offset,0,0.000000
timestamp,0,0.000000
site_id,0,0.000000


In [9]:
weather_train_df = weather_train_df.groupby('site_id').apply(
    lambda group: group.interpolate(limit_direction='both'))

weather_test_df = weather_test_df.groupby('site_id').apply(
    lambda group: group.interpolate(limit_direction='both'))

In [10]:
print("Train")
display(check_missing(weather_train_df))
print("Test")
display(check_missing(weather_test_df))

Train


,Total,Percent missing
precip_depth_1_hr,26273,18.796906
cloud_coverage,17228,12.325700
sea_level_pressure,8755,6.263728
offset,0,0.000000
wind_speed,0,0.000000
wind_direction,0,0.000000
dew_temperature,0,0.000000
air_temperature,0,0.000000
timestamp,0,0.000000
site_id,0,0.000000


Test


,Total,Percent missing
precip_depth_1_hr,51807,18.686495
cloud_coverage,33146,11.955577
sea_level_pressure,17241,6.218732
offset,0,0.000000
wind_speed,0,0.000000
wind_direction,0,0.000000
dew_temperature,0,0.000000
air_temperature,0,0.000000
timestamp,0,0.000000
site_id,0,0.000000


# Create features (pandas)

In [11]:
################################################ Add buildings features
train_df = train_df.merge(metadata_df, on='building_id', how='left')
test_df  = test_df.merge(metadata_df,  on='building_id', how='left')

del metadata_df
gc.collect()

train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN


In [12]:
################################################ Add weather features
train_df = train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')
test_df  = test_df.merge(weather_test_df,   on=['site_id', 'timestamp'], how='left')

del weather_train_df, weather_test_df
gc.collect()

train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,19.40625,4.0,19.40625,0.0,1020.0,0.0,0.0,5.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,19.40625,4.0,19.40625,0.0,1020.0,0.0,0.0,5.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,19.40625,4.0,19.40625,0.0,1020.0,0.0,0.0,5.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,19.40625,4.0,19.40625,0.0,1020.0,0.0,0.0,5.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,19.40625,4.0,19.40625,0.0,1020.0,0.0,0.0,5.0


In [13]:
def add_timeSeries_feats(df, datetime_name):
    #df['year']    = df[datetime_name].dt.year       # Extract year
    df['month']    = df[datetime_name].dt.month      # Extract month
    df['monthday'] = df[datetime_name].dt.day        # Extract day of month
    df['weekday']  = df[datetime_name].dt.day_name() # Extract weekday name
    df['hour']     = df[datetime_name].dt.hour       # Extract hour
    #df['minute']  = df[datetime_name].dt.minute     # Extract minute
    
    #df.drop(datetime_name, axis=1, inplace=True)
    
################################################ Add time series features
add_timeSeries_feats(train_df, "timestamp")
add_timeSeries_feats(test_df,  "timestamp")
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,offset,month,monthday,weekday,hour
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,19.40625,...,19.40625,0.0,1020.0,0.0,0.0,5.0,1,1,Friday,0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,19.40625,...,19.40625,0.0,1020.0,0.0,0.0,5.0,1,1,Friday,0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,19.40625,...,19.40625,0.0,1020.0,0.0,0.0,5.0,1,1,Friday,0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,19.40625,...,19.40625,0.0,1020.0,0.0,0.0,5.0,1,1,Friday,0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,19.40625,...,19.40625,0.0,1020.0,0.0,0.0,5.0,1,1,Friday,0


# Optimize

In [14]:
train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)
gc.collect()

Mem. usage decreased to 1291.73 Mb (28.7% reduction)
Mem. usage decreased to 2664.32 Mb (28.7% reduction)


20

# Drop all NaN rows which are generated by timestamp alignment

In [15]:
check_missing(train_df)

,Total,Percent missing
floor_count,16709167,82.652772
year_built,12127645,59.990033
precip_depth_1_hr,1749549,8.654236
sea_level_pressure,882646,4.366055
cloud_coverage,580004,2.869020
offset,103451,0.511726
wind_speed,103451,0.511726
wind_direction,103451,0.511726
dew_temperature,103451,0.511726
air_temperature,103451,0.511726


In [17]:
train_df = train_df.loc[~(
    train_df['air_temperature'].isnull() & 
    train_df['cloud_coverage'].isnull() &
    train_df['dew_temperature'].isnull() &
    train_df['precip_depth_1_hr'].isnull() &
    train_df['sea_level_pressure'].isnull() &
    train_df['wind_direction'].isnull() &
    train_df['wind_speed'].isnull() &
    train_df['offset'].isnull())]

In [18]:
check_missing(train_df)

,Total,Percent missing
floor_count,16618674,82.627972
year_built,12105206,60.187030
precip_depth_1_hr,1646098,8.184392
sea_level_pressure,779195,3.874154
cloud_coverage,476553,2.369419
meter,0,0.000000
timestamp,0,0.000000
meter_reading,0,0.000000
site_id,0,0.000000
primary_use,0,0.000000


# Select features (pandas -> numpy)

In [19]:
building_cat_feats = ["building_id", "site_id", "meter", "primary_use"]
building_num_feats = ["square_feet", "year_built", "floor_count"]

weather_num_feats  = ["air_temperature", "cloud_coverage", "dew_temperature", "precip_depth_1_hr",
                      "sea_level_pressure", "wind_direction", "wind_speed"]

datetime_cat_feats = ["month", "monthday", "weekday", "hour"]

cat_feats = building_cat_feats + datetime_cat_feats
num_feats = building_num_feats + weather_num_feats

for c in cat_feats:
    train_df[c] = train_df[c].astype('category')
    
feats  = cat_feats + num_feats
target = ["meter_reading"]


x = train_df[feats] # train_df[feats].values
y = np.log1p(train_df[target].values)
test = test_df[feats] # test_df[feats].values

# Missings (numpy)

In [9]:
#imputer = SimpleImputer(strategy='mean', missing_values=np.nan, add_indicator=False)
#imputer.fit(x)

#x    = imputer.transform(x)
#test = imputer.transform(test)

# Boosting

In [20]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.25,
            'subsample_freq': 1,
            'learning_rate': 0.4,
            'num_leaves': 20,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            }

params2 = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.4,
            'subsample_freq': 1,
            'learning_rate': 0.25,
            'num_leaves': 31,
            'feature_fraction': 0.8,
            'lambda_l1': 1,
            'lambda_l2': 1
            }

In [ ]:
folds = 4
seed = 42

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

models = []
for fold,(train_idx, val_idx) in enumerate(tqdm(kf.split(train_df, train_df['building_id']),
                                               total=folds)):
    x_train,x_val = x.iloc[train_idx], x.iloc[val_idx]
    y_train,y_val =      y[train_idx],      y[val_idx]

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_val   = lgb.Dataset(x_val,   y_val)
    
    gbm = lgb.train(
        params, lgb_train, num_boost_round=1000, valid_sets=(lgb_train, lgb_val),
        early_stopping_rounds=100, verbose_eval=100)
    
    models.append(gbm)
    log_preds_val = gbm.predict(x_val)
    print(f'Fold: {fold} VAL RMSLE: {np.sqrt(mean_squared_error( y_val, log_preds_val))}')